In [1]:
import pandas as pd
import numpy as np
from src.api_osmr import get_matrix, parse_coordinates
from src.routing import optimize_routes, plot_route

# Reading data

In [2]:
# Read the places data from the CSV file
places = pd.read_csv("data/input/pos.csv")
places.head()

,city,lat,lng,name,province,street
0,Alcanar,40.593153,0.561053,Bar Montsia Mar,Tarragona,"Carrer Tarongina, 2"
1,Alcanar,40.542112,0.481537,Bar Paella,Tarragona,"Carrer del Canonge Matamoros, 7"
2,Alcanar,40.543882,0.477914,Restaurant l’Esquella D’en Gaudí,Tarragona,"Carrer de l'Arquitecte Gaudí, 26"
3,Alcanar,40.528650,0.509533,Citrus del Tancat,Tarragona,C N-340 Km 1059 (Sol de Riu
4,Alcanar,40.569059,0.538400,Bar km.1065,Tarragona,"Carrer a Platja Daurada, 2"


In [3]:
# Calculate points of sales centroids for each city
# Group the places data by "city" and calculate the median latitude and longitude for each city
cities = places.groupby(["city"])[["lat", "lng"]].median().reset_index()
cities.head()

,city,lat,lng
0,Alcanar,40.543398,0.481130
1,Amposta,40.708700,0.579551
2,Deltebre,40.719954,0.734037
3,L'Aldea,40.750078,0.622673
4,La Ràpita,40.618336,0.591224


# Most optimal route for visiting all cities

In [4]:
# Extract the list of cities from the "cities" DataFrame
cities_list = cities.city

# Parse coordinates for the cities
coordinates = parse_coordinates(cities)

# Get the distance matrix for the cities using the OSRM API
response_distances = get_matrix(coordinates)

# Convert the distances to integer values (required by ortools)
distances = np.array(response_distances["distances"]).astype(int)

# Set the starting city for the route (change if needed)
starting_city = "Ulldecona"

# Find the index of the starting city in the cities_list
depot_index = int(cities_list.index[cities_list == starting_city][0])

# Optimize the route using the distance matrix and starting city index
idx_route = optimize_routes(distances, depot_index, list(cities_list))

Route for vehicle:
 Ulldecona -> Alcanar -> Les Cases d'Alcanar -> La Ràpita -> Amposta -> Deltebre -> L'Aldea -> Tortosa -> Ulldecona
Route distance: 107.8km



In [5]:
# Create a DataFrame with the best route
df_best_route = cities.iloc[idx_route]

# Get the coordinates of the best route
coordinates = list(zip(df_best_route["lat"], df_best_route["lng"]))

# Plot the best route on a Folium map
my_map = plot_route(coordinates, dist=30000, zoom_start=11)
my_map

In [6]:
# saving interactive map in html
my_map.save('maps/map_cities_route.html')

# Most optimal route for visiting 10 points of sales for each city

In [7]:
# Iterate over each unique city in the places data
for city in places.city.unique():
    print(f"Planning route for {city}")

    # Select the first 10 points of sales for the current city
    df_city = places[places.city == city][:10].reset_index(drop=True)

    # Parse coordinates for the selected points of sales
    coordinates = parse_coordinates(df_city)

    # Get the distance matrix for the selected points of sales using the OSRM API
    response_distances = get_matrix(coordinates)

    # Convert the distances to integer values (required by ortools)
    distances = np.array(response_distances["distances"]).astype(int)

    # Set the names of the points of sales for labeling the route
    pos_names = list(df_city.name)

    # Setting the farthest point as the start point
    # Calculate the median tuple for the distance matrix
    coordinates_tuples = np.array(response_distances["distances"]).astype(int)
    median_tuple = tuple(np.median(np.array(coordinates_tuples), axis=0))

    # Calculate the Euclidean distance of each tuple from the median tuple
    distances_tuples = [
        np.linalg.norm(np.array(t) - np.array(median_tuple)) for t in coordinates_tuples
    ]

    # Find the index of the tuple with the maximum distance
    farthest_index = max(enumerate(distances_tuples), key=lambda x: x[1])[0]

    # Optimize the route using the distance matrix and farthest point index as the starting point
    best_route = optimize_routes(distances, farthest_index, pos_names)[:-1]

    # Save the best route to a CSV file with the city name in the output folder
    df_city.name[best_route].reset_index(drop=True).to_csv(
        f"data/output/{city}_routing.csv"
    )

    # Plot the best route on the map for the city "Ulldecona" (first city in the iteration)
    if city == "Ulldecona":
        df_best_route = df_city.iloc[best_route]
        coordinates = list(zip(df_best_route["lat"], df_best_route["lng"]))
        my_map = plot_route(coordinates, dist=2000)

# Display the final Folium map with the best route for the city "Ulldecona"
my_map

Planning route for Alcanar
Route for vehicle:
 Restaurante Club de Tenis Serramar -> Bar Montsia Mar -> Bar km.1065 -> Restaurante L’Estona -> Juan Pedro Lara Márquez -> Restaurant l’Esquella D’en Gaudí -> Bar Paella -> Sofía Alonso Beltrán -> Bar Moreno, C.B -> Citrus del Tancat -> Restaurante Club de Tenis Serramar
Route distance: 27.7km

Planning route for Amposta
Route for vehicle:
 Bar Residència L’Ullal Apasa -> Bar Restaurant Bon Gust -> Basar Amposta -> Prince Doner Kebab | Soylent Green -> Les Palmeres -> Bar Simpatica -> Amore Pizza - Doner Kebab | Soylent Green -> Únic Restaurant | Imperio Suichy -> Bar Fidel -> Bar Restaurant La Nova Torreta -> Bar Residència L’Ullal Apasa
Route distance: 4.7km

Planning route for Deltebre
Route for vehicle:
 Cafetería Bar Restaurante Lo Mirador -> Restaurant l'Arròs -> Restaurant Lo Transbordador -> Ristorante La Dolce Vita -> Cafeteria Restaurant Casa Nicanor -> Bar Restaurant de tots edades N40 -> Carrer Lovas Companies -> Restaurante @ 

In [8]:
# saving interactive map in html
my_map.save('maps/map_ulldecona_route.html')